In [1]:
%load_ext autoreload
%autoreload 2

import os
import io
import sys
import re
import csv
import pandas as pd
from tqdm import tqdm_notebook, tnrange, notebook
from datetime import datetime
import function_library3 as fctlib3

In [110]:
re_From = re.compile(r'^\s?From\:\s?(.*)')
re_From2 = re.compile(r'^F.S. 119.071 - Security .*From\:\s(.*)')
re_Sent = re.compile(r'^Sent\:\s?(.*)')
re_Sent2 = re.compile(r'^Date\:\s?(.*)')
re_To = re.compile(r'^To\:\s?(.*)')
re_CC = re.compile(r'^C[Cc]\:\s?(.*)')
re_Subject = re.compile(r'^Sub\s?j\s?ect\:\s?(.*)')
re_Subject2 = re.compile(r'^Title\:\s?(.*)')
re_Attachments = re.compile(r'^Attachments\:\s?(.*)')
re_Importance = re.compile(r'Importance\:\s?(.*)')

re_line_separator = re.compile(r'(-+\s*?Original\s+\w+\s*?-+\s*)(\w)')
                               
re_page = re.compile(r'^Page\s\d+$')
re_page2 = re.compile(r'^Page\s\d+\sof\s\d+$')
re_page3 = re.compile(r'^\d+\-\d+$')

re_tail = re.compile(r'^Sent\sfrom\s.*\siPad')
re_tail2 = re.compile(r'^Sent\sfrom\s.*\siPhone')
re_tail3 = re.compile(r'^On\s\w+\,\s\w+\swrote\:')
re_tail4 = re.compile(r'^On\s.+\,\s[0-9]{4},\sat\s.+\,.+')
re_tail5 = re.compile(r'^Begin\sforwarded\smessage\:')

tails = ['Sent from my iPhone',
         "Sent from Paige's iPad",
        'On \w+\, \w+ wrote:',
        'Text messages of Karen Jumonville To/From Adam Corey',
         'On Jun 1, 2013, at 8:21 AM, "Favors Thompson, Anita"',
        'Begin forwarded message:']

re_lowercase = re.compile(r'^[a-z].*')

In [113]:
a = "-----Original    Message ------ Subject"
print(re_line_separator.match(a))
line = re_line_separator.sub('r\2', a)
# print(re_line_separator.groupindex(0))
print(line)


<re.Match object; span=(0, 33), match='-----Original    Message ------ S'>
rubject


In [3]:
# test
re_tail3.findall("Sent from K's iPhone")
re_tail5.findall(tails[5])

['Begin forwarded message:']

In [114]:
def split_emails(inputpath, outputpath, file_name, filecount=None):
    #     f_in = io.open(inputpath, encoding='utf8',errors='ignore')
    f_in = open(inputpath, encoding="utf8")

    if filecount == None:
        # len(os.listdir(outputpath)) # files are named by numbers
        filecount = 0
        
    email = []
    #     email.append(f_in.readline().strip('\f'))
    
    try:
        for line_no, line in enumerate(f_in.readlines()):
            if re_line_separator.match(line):
                pass
            
            line = line.strip("\f").strip(" ")
            # print("line: ",  line)
            # m1 = re_From.match(line)
            # print("m1: ", m1)
            # m2 = re_From2.match(line)
            # print("m2: ", m2)
            if re_From.match(line) or re_From2.match(line) or re_line_separator.match(line):
                # Start new email
                if re_line_separator.match(line):
                    line = re_line_separator.sub(r'\2', line)
                    print("separator line: ", line)
                if email != []:
                    # print("before open, filecount: ", filecount+1, ", file_name :", file_name, ", line_no: ", line_no)
                    # print("outputpath: ", outputpath)
                    with open(
                        outputpath + "/" + "id=" + str(filecount) + "_" + "fn=" + str(file_name) + 
                        "_" + "ln=" + str(line_no) + ".txt",
                        "w+",
                        encoding="utf-8",
                    ) as f_out:
                        # print("outputpath: ", outputpath)
                        for l in email:
                            f_out.write(l)
                email = []
                filecount = filecount + 1
            email.append(line)
            # print("email append, email: ", email)
    except:
        print("split_emails: except")   # Why is except called?
        pass
    f_in.close

    return filecount

In [115]:
# Read mails from text files (translated from pdf)
# Note: terminate paths with a forward slash
inputpath = './email_txt/downloaded_from_data_tallahassee/'  
inputpath = './email_txt/test_files/'  
outputpath = './split_txt/' 
outputpath = './test_split_txt/'
# inputpath = './redacted_email_txt/'
# outputpath = './redacted_split_txt/' 
# inputpath = './unredacted_email_txt/'
# outputpath = './unredacted_split_txt/' 
# inputpath = './old_email_txt/'
# outputpath = './old_split_txt/' 
filecount = 0
for inputfile in notebook.tqdm(os.listdir(inputpath)):
#     print('processing '+inputpath+inputfile)
    ## JOEY: why subtract 1 from filecount below? <<<<<<
    print(inputfile)
    #filecount = split_emails(inputpath+inputfile, outputpath, inputfile, filecount-1)  # ORIG
    filecount = split_emails(inputpath+inputfile, outputpath, inputfile, filecount)
    if filecount > 5:
        print("filecount: ", filecount)
        break
    

  0%|          | 0/1 [00:00<?, ?it/s]

gordon.txt
separator line:  Subject: Re: per Bobby Burleson



In [40]:
def correct_line(inputpath, outputpath=''):
    """
    What does this file do? 
    """
    if outputpath=='':
        outputpath = inputpath
    new_file = []
    with open(inputpath, 'r', encoding='utf8') as f:
        flag = 0
        body = []
        for line in f.readlines():
            # remove formfeeds
            line = re.sub('\f', '', line).strip()
            
            if re_tail.match(line) or re_tail2.match(line) or re_tail3.match(line) or re_tail4.match(line) or re_tail5.match(line):
                break
            if re_page.match(line) or re_page2.match(line) or re_page3.match(line):
                continue
            if re_From.match(line) or re_From2.match(line) or re_Sent.match(line) or re_Sent2.match(line) or re_To.match(line) or re_CC.match(line):
                if body != []:
                    temp_line = ' '.join(body)
                    new_file.append(temp_line)
                body = []
                body.append(line)
            elif re_Subject.match(line):
                flag = 1
                temp_line = ' '.join(body)
                body = []
                new_file.append(temp_line)
                body.append(line)
            elif re_Attachments.match(line) and flag == 1:
                flag = 2
                temp_line = ' '.join(body)
                body = []
                body.append(line)
                new_file.append(temp_line)
            elif re_Attachments.match(line) and flag == 0:
                flag = 3
                temp_line = ' '.join(body)
                body = []
                new_file.append(temp_line)
                body.append(line)
#                 print(line)
#                 print(re_Attachments.match(line))
#                 print( re_Attachments.findall(line))
                print('no title but attachment in file'+inputpath)
#                 return
            else:
                body.append(line)
            
        if flag == 0:
            flag2 = 0
            new_file.append(body[0])
            for line in body[1:]:
                if ('<' in line or '>' in line or '(' in line or ')' in line or ',' in line or ';' in line or line == '') and flag2 == 0:
                    new_file[-1] = new_file[-1] + ' ' + line
                elif flag2 == 1:
                    new_file.append(line)
                else:
                    flag2 = 1
                    new_file.append('Subject: ' + line)
        else:
            for line in body:
                new_file.append(line)
        

    with open(outputpath, 'w+', encoding = 'utf-8') as f_out:
        for line in new_file:
            f_out.write(line + '\n')

In [49]:
inputpath = './split_txt/' 
# inputpath = './redacted_split_txt/' 
# inputpath = './unredacted_split_txt/' 
#inputpath = './old_split_txt/' 
inputpath = './test_split_txt/'

""" 
GE: Somewhere, the lines

From: Gayle Webb <gagawebb@comcast.net>
To: "Powell, Jr., Rudy" <Rudy.Powell@dot.state.fl.us>
CC:
Date: Tue, September 22, 2015 12:14 PM

Hi Cris,
I received a request to see if it would
be possible to prune up what appear to


got converted to: 


From: Gayle Webb <gagawebb@comcast.net>
To: "Powell, Jr., Rudy" <Rudy.Powell@dot.state.fl.us>
CC:
Date: Tue, September 22, 2015 12:14 PM  Hi Cris,
Subject: I received a request to see if it would
be possible to prune up what appear to

    
"""

abnormal_email_id = []
for name in tqdm_notebook(os.listdir(inputpath)):
    #filenm = inputpath + '/' + name # not the best way
    filenm = os.path.join(inputpath, name)  # works on windows, 
    try:
        correct_line(filenm)
    except:
        abnormal_email_id.append(int(name[:-4]))
        print(name)
#         os.remove(inputpath+'/'+name)

<ipython-input-49-2c12a2b30f6f>:34: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for name in tqdm_notebook(os.listdir(inputpath)):


  0%|          | 0/2 [00:00<?, ?it/s]

In [49]:
# test
# correct_line('./unredacted_split_txt/66012.txt')

no title but attachment in file./unredacted_split_txt/66012.txt


In [42]:
def get_headers(inputpath, outputpath='', file_name='x'): 
    dictionary = {}

    with open(inputpath, 'r',encoding='utf8') as f:
        list_from = []
        list_sent = []
        list_to = []
        list_cc = []
        list_subject = []
        list_attachments = []
        list_body = []
        list_importance = []
        for line in f.readlines():
            #\f is a special character (dot)
            line = re.sub('\f', '', line)
            line = line.strip()
            if re_From.match(line):
                list_from.append(re_From.findall(line)[0])
            elif re_Sent.match(line) or re_Sent2.match(line):
                if re_Sent.match(line):
                    list_sent.append(re_Sent.findall(line)[0])
                else:
                    list_sent.append(re_Sent2.findall(line)[0])

            elif re_To.match(line):
                list_to.append(re_To.findall(line)[0])
            elif re_CC.match(line):
                list_cc.append(re_CC.findall(line)[0])
            elif re_Subject.match(line) or re_Subject2.match(line):
                if re_Subject.match(line):
                    list_subject.append(re_Subject.findall(line)[0])
                else:
                    list_subject.append(re_Subject2.findall(line)[0])

            elif re_Attachments.match(line):
                list_attachments.append(re_Attachments.findall(line)[0])
            elif re_Importance.match(line):
                list_importance.append(re_Importance.findall(line)[0])
            else:
                # state what this does.
                line = re.sub('[^A-Za-z0-9\s\(\)\/\\\&\?\!\,\.\-\@\<\>\:\+\%\;]', '', line)
                list_body.append(line)
                
    # print("file_name: ", file_name)
    splits = file_name.split("_")
    Id = splits[0][3:]
    name = splits[1][3:]
    line_no = splits[2][3:-3]
    # print(f"Id: {Id}, name: {name}, line_no: {line_no}")
    dictionary['Id'] = Id # file name is id.txt
    dictionary['From'] = list_from
    dictionary['Sent'] = list_sent
    dictionary['To'] = list_to
    dictionary['CC'] = list_cc
    dictionary['Subject'] = list_subject
    dictionary['Attachments'] = list_attachments
    dictionary['Body'] = list_body
    dictionary['Importance'] = list_importance
    dictionary['FromFile'] = name
    dictionary['FileLineNo'] = line_no
    
    return dictionary

In [43]:
inputpath = './split_txt/' 
# inputpath = './redacted_split_txt/' 
# inputpath = './unredacted_split_txt/' 
# inputpath = './old_split_txt/' 
l = []
for i, name in enumerate( tqdm_notebook(os.listdir(inputpath))):
    if name != '.DS_Store':
        # print("name: ", name)
        l.append(get_headers(inputpath+'/'+name, file_name=name))
    #if i == 5: break
len(l)

<ipython-input-43-1c416b10e5dc>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, name in enumerate( tqdm_notebook(os.listdir(inputpath))):


  0%|          | 0/338 [00:00<?, ?it/s]

338

In [50]:
# It would be easier to create a pandas array and then do  df.write_csv("filenm", index=0)
csv_columns = ['Id','From','Sent','To','CC','Subject','Attachments','Importance','Body','FromFile','FileLineNo']
with open('./output.csv', 'w',encoding='utf8') as csvfile:
# with open('./redacted_output.csv', 'w',encoding='utf8') as csvfile:
# with open('./unredacted_output.csv', 'w',encoding='utf8') as csvfile:
# with open('./old_output.csv', 'w',encoding='utf8') as csvfile:

    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for data in l:
        writer.writerow(data)

In [93]:
df = pd.read_csv('output.csv')
# df_redacted = pd.read_csv('redacted_output.csv')
# df_unredacted = pd.read_csv('unredacted_output.csv')
# df_old = pd.read_csv('old_output.csv')

df.head()
# df_redacted.head()
# df_unredacted.head()
# df_old.head()

,Id,From,Sent,To,CC,Subject,Attachments,Importance,Body,FromFile,FileLineNo
0,167,['Chris Dudley'],"['Monday, June 13, 2016 11:27 AM']",['Paige Carter-Smith'],['Claudia Davant; DIA Scott Dick; Cynthia Tunn...,['Re: TDIA - Board Meeting'],[],[],"['', 'I will be.']",23-11-PCSgmail2014-2017.txt,10350.0
1,142,['Elias Mathes TDIA'],"['Friday, June 03, 2016 7:12 AM']",['Allison Behrman'],['Paige Carter-Smith'],['TDIA Co-Sponsorship Applications'],['Co-Sponsorship Application_2016 Downtown Get...,[],"['Sundown Summer Concert Series SIGNED.pdf', '...",23-11-PCSgmail2014-2017.txt,8514.0
2,41,['Mallory Izbicki'],"['Thursday, March 19, 2015 11:21 AM']","[""'Bellamy, Mike (Fire Dept)'; 'Jay Revell'; R...",['Nicola Barnack'],['Oxygen Ball Cocktail Hour Wrap Up'],[],[],"['Good Morning Celebrities!', 'We are planning...",23-11-PCSgmail2014-2017.txt,1840.0
3,256,[''],[''],[''],[],[''],[],[],"['', 'Paige Carter-Smith <paigecartersmith@gma...",23-11-PCSgmail2014-2017.txt,16147.0
4,201,['Fran Conaway [mailto:franconaway32312@gmail....,"['Sunday, July 24, 2016 8:42 PM', 'Sat, Jul 23...","['Whitaker, Angie; Paigecartersmith@gmail.com'...",[],"[""Fwd: An Invitation to SeeTallahassee in a Ne...",[],[],"['<Angie.Whitaker@talgov.com>, Gil.Ziffer@talg...",23-11-PCSgmail2014-2017.txt,12461.0


In [99]:
new_dates = fctlib3.process_dates_new(df)

  0%|          | 0/338 [00:00<?, ?it/s]

dt1:   June 13 2016 11:27 AM
dt1:   June 03 2016 7:12 AM
dt1:   March 19 2015 11:21 AM
dt1:  
dt1:   July 24 2016 8:42 PM
dt1:   March 31 2015 2:01 PM
dt1:   October 07 2016 8:21 AM
dt1:   June 08 2016 9:56 AM
dt1:   July 22 2016 4:58 PM
dt1:   July 27 2016 10:43 AM
dt1:   March 31 2015 2:01 PM
dt1:   October 16 2014 7:53 PM
dt1:   September 15 2015 11:20 AM
dt1:  
dt1:   May 27 2015 1:28 PM
dt1:  
dt1:   July 22 2016 4:58 PM
dt1:   June 13 2016 11:58 AM
dt1:   February 18 2015 10:15 AM
dt1:   June 09 2016 11:18 PM
dt1:   September 15 2015 11:20 AM
dt1:   August 02 2016 11:41 AM
dt1:  August 1 2015 1:59:26 AM EDT
dt1:   May 27 2015 1:28 PM
dt1:   April 20 2017 12:26 PM
dt1:   June 14 2016 3:43 PM
dt1:   January 05 2017 10:56 AM
dt1:  
dt1:   October 12 2016 2:40 PM
dt1:   June 09 2016 11:08 AM
dt1:   July 22 2016 3:38 PM
dt1:   September 07 2016 7:03 PM
dt1:   February 20 2015 9:50 AM
dt1:   October 07 2016 10:30 AM
dt1:   September 22 2015 12:14 PM Hi Cris 
dt1:   April 20 2017 5:08 P

In [92]:
df["Sent2"] = new_dates
df = df.sort_values(by="Sent2")
# df_redacted = df.copy()
df_old = df.copy()

# df.to_csv("output2_time_identified.csv", header=True)

  0%|          | 0/338 [00:00<?, ?it/s]

dt:  Tue, September 22, 2015 12:14 PM  Hi Cris,
dt1:  , September 22, 2015 12:14 PM  Hi Cris,
dt:  Friday, July 22, 2016 3:38 PM
dt1:  , July 22, 2016 3:38 PM
dt:  Monday, July 18, 2016 2:39 PM
dt1:  , July 18, 2016 2:39 PM
dt:  Monday, July 18, 2016 2:39 PM
dt1:  , July 18, 2016 2:39 PM
dt:  Wednesday, July 27, 2016 10:43 AM
dt1:  , July 27, 2016 10:43 AM
dt:  Monday, July 18, 2016 2:39 PM
dt1:  , July 18, 2016 2:39 PM
dt:  Sunday, July 10, 2016 2:26 PM
dt1:  , July 10, 2016 2:26 PM
dt:  Tuesday, July 26, 2016 10:07 AM
dt1:  , July 26, 2016 10:07 AM
dt:  Monday, August 01, 2016 9:20 AM
dt1:  , August 01, 2016 9:20 AM
dt:  Friday, July 01, 2016 12:32 PM
dt1:  , July 01, 2016 12:32 PM
dt:  Sunday, July 24, 2016 10:57 PM
dt1:  , July 24, 2016 10:57 PM
dt:  Sunday, July 24, 2016 10:52 PM
dt1:  , July 24, 2016 10:52 PM
dt:  Sunday, July 24, 2016 8:42 PM
dt1:  , July 24, 2016 8:42 PM
dt:  Sunday, July 24, 2016 8:41 PM
dt1:  , July 24, 2016 8:41 PM
dt:  Friday, July 22, 2016 5:00 PM
dt1:  , 

In [84]:
date2 = pd.to_datetime(df['Sent2'].values.tolist())
idx = date2 > datetime(2000,1,1)
df2 = df.iloc[idx]
df2 = df2.reset_index(drop=True)
# df2.to_csv('output3_with_good_time.csv')

In [69]:
df2

,Id,From,Sent,To,CC,Subject,Attachments,Importance,Body,FromFile,FileLineNo,Sent2
0,1,"['Bowen, Dan']","['Monday, April 21, 2014 5:28 PM']","[""'electricalsrvs@aol.com'""]","['Ash, Richard']",['maddox building'],[],[],"['', 'Mr. Howell, per our conversation this af...",23-11-PCSgmail2014-2017.txt,57.0,2014-04-21 17:28:00
1,3,"['Bowen, Dan']","['Monday, April 21, 2014 5:28 PM']","[""'electricalsrvs@aol.com'""]","['Ash, Richard']",['maddox building'],[],[],"['', 'Mr. Howell, per our conversation this af...",23-11-PCSgmail2014-2017.txt,113.0,2014-04-21 17:28:00
2,0,"['Bowen, Dan']","['Tuesday, April 22, 2014 3:34 PM']","[""'paigecartersmith@gmail.com'""]","[""'electricalsrvs@aol.com'; Ash, Richard""]",['902 NORTH ADAMS ST'],['maddox building (5.37 KB)'],[],"['', 'Ms. Smith, Mr. Eddie Johnson had Ms. Cam...",23-11-PCSgmail2014-2017.txt,26.0,2014-04-22 15:34:00
3,2,"['Bowen, Dan']","['Thursday, May 01, 2014 4:52 PM']","[""'electricalsrvs@aol.com'; paigecartersmith@g...","['Ash, Richard; Drose, Tina; Lockwood, Kenny']",['FW: maddox building'],[],[],"['', 'Mr. Howell, The email below was the last...",23-11-PCSgmail2014-2017.txt,81.0,2014-05-01 16:52:00
4,4,['Gail Stansberry-Ziffer'],"['Wednesday, May 07, 2014 9:33 PM']","['Karen Wendland; Vince Dix, Ph.D.; Jerome Nov...",[],['Hope you can join us'],[],[],"['', 'Tomorrow morning Gil and I will be on-ai...",23-11-PCSgmail2014-2017.txt,155.0,2014-05-07 21:33:00
...,...,...,...,...,...,...,...,...,...,...,...,...
300,309,['Paige Carter-Smith'],"['Thursday, April 20, 2017 12:26 PM']",['edward.kring@talgov.com'],['TDIA Cell'],[''],['Trolley award Entry (2).docx'],[],"['', 'Research/Situation Analysis: Oh what joy...",23-11-PCSgmail2014-2017.txt,18392.0,2017-04-20 12:26:00
301,306,"['Kring, Edward']","['Thursday, April 20, 2017 1:26 PM']",['Paige Carter-Smith'],['TDIA Cell'],['RE:'],['Trolley award Entry 2017.docx'],[],['Gracias! Please send me any additions to the...,23-11-PCSgmail2014-2017.txt,18152.0,2017-04-20 13:26:00
302,308,['Paige Carter-Smith'],"['Thursday, April 20, 2017 5:08 PM']",['edward.kring@talgov.com'],[],[''],['Trolley award Entry 2017.docx'],[],"['', 'Research/Situation Analysis: Oh what joy...",23-11-PCSgmail2014-2017.txt,18307.0,2017-04-20 17:08:00
303,310,"['German, Michael']","['Tuesday, May 02, 2017 9:16 AM']","[""Amanda Sauer; catkins@crowncars.bm; jan.bubs...",[],['E-mail'],[],[],"['Hello all,', 'We are experiencing some e-mai...",23-11-PCSgmail2014-2017.txt,18415.0,2017-05-02 09:16:00


In [66]:
df2.drop_duplicates(subset=['From','Sent2'],keep='first',inplace = True)
df2 = df2.reset_index(drop=True)
# df_redacted = df2.copy()
df_old = df2.copy()

# df2.to_csv('output4_without_dups.csv')
# df_redacted.to_csv('redacted_emails.csv')
# df_unredacted.to_csv('unredacted_emails.csv')
df_old.to_csv('old_emails. csv')

df2

,From,Sent,To,CC,Subject,Attachments,Importance,Body,Sent2
0,"['Heiter, Deborah [mailto:Deborah.Heiter@talgo...","['Friday, January 20, 2012 4:41 PM']","[""Behenna, Bill; Branch, Cathy; Burn, Paula; C...",[],['Economic Development Target Issue Meeting'],[],[],"['', 'Deborah L. Heiter', 'Administrative Spec...",2012-01-20 16:41:00
1,['Beth Kirkland [mailto:bkirkland@taledc.com]'],"['Monday, January 23, 2012 9:32 AM']","[""'llaubscher@eflorida.com'""]","[""brook.pace@deo.myflorida.com; Kim Rivers (ki...",['City of Tallahassee meeting regarding SBCI'],[],[],"['Louis,', 'Thank you for meeting with us in F...",2012-01-23 09:32:00
2,"['Wiebler, Brian T.']","['Monday, February 27, 2012 10:04 AM']",[],[],['Rural Density'],[],[],[''],2012-02-27 10:04:00
3,"['Beaudoin, Mark']","['Monday, March 05, 2012 11:33 AM']","['Corey, Adam']","['English, Jim; Coe, Torn; Bell?ower, Adrienne']",['Sidewalk Cafe Agreement'],[],[],"['', 'Adam, thanks for the update regarding th...",2012-03-05 11:33:00
4,"['Wiebler, Brian T.']","['Tuesday, March 27, 2012 3:59 PM']","['Wiebler, Brian T.']",[],['Rural Density Comprehensive Plan Text Amendm...,[],[],"['', 'This is a reminder of the open house eve...",2012-03-27 15:59:00
...,...,...,...,...,...,...,...,...,...
13085,"['Campbell, Stacey']","['Friday, June 23, 2017 1:27 PM']",['Ryan Grindler'],"['Donahoe, Judy']",['RE: Request: Construction Warranties'],[],[],"['Perfect, thank you.']",2017-06-23 13:27:00
13086,['Gil Ziffer [mailto:gil@ziffberry.com]'],"['Wednesday, June 28, 2017 3:32 PM']","['Branch, Cathy <Cathy.Branch@talgov.com>']",[],['Fwd: 2013 AIP Forum White Paper_final (3).pdf'],[],[],"['', 'Gil Ziffer', '(850) 509-7886']",2017-06-28 15:32:00
13087,['Gil Ziffer [mailto:gil@ziffberry.com]'],"['Thursday, June 29, 2017 4:41 AM']","['Branch, Cathy <Cathy.Branch@talgov.com>']",[],['Fwd: 2013 AIP Forum White Paper_final (3).pdf'],[],[],"['', 'Subpoena']",2017-06-29 04:41:00
13088,['Gil Ziffer [mailto:gil@ziffberry.com]'],"['Thursday, June 29, 2017 5:35 AM']","['Branch, Cathy <Cathy.Branch@talgov.com>']",[],['Fwd: RE: Fwd:'],[],[],"['', 'Subpoena']",2017-06-29 05:35:00
